In [6]:
import pymysql
import psycopg2
import pandas as pd
from secret.auth_data import *
import secret.auth_data_postgreSQL as p
import telegram_constants as c
import parsing_LTE as table

import sys
import os

import parsing_LTE as table

In [16]:
try:
    connection_mysql = pymysql.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database,
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor
    )
    print('successfully connected MySQL ...')
except Exception as ex:
    print(f"Connection refused MySQL ... \n{ex}")

try:
    connection = psycopg2.connect(
        host=p.host,
        port=p.port,
        user=p.user,
        password=p.password,
        database=p.database
    )
    print('successfully connected postgreSQL ...')
except Exception as ex:
    print(f"Connection refused postgreSQL ... \n{ex}")

successfully connected MySQL ...
Connection refused postgreSQL ... 
could not connect to server: Connection refused
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?



In [14]:
try:
    with connection.cursor() as cursor:

        for city_key in c.SET_CITIES:
            city_key = c.TRANSLATE_CITIES[city_key]
            for type_key in c.SET_TYPES:

                df = table.view(city_key, type_key, connection_mysql, 'all')

                # Создание таблицы (если она еще не создана)
                table_name = f"{city_key}_{type_key}"
                cursor.execute(f"""
                    CREATE TABLE IF NOT EXISTS {table_name} (
                        date DATE,
                        day1 INT,
                        day2 INT,
                        day3 INT,
                        day4 INT,
                        day5 INT,
                        day6 INT,
                        day7 INT,
                        day8 INT,
                        day9 INT,
                        day10 INT,
                        night1 INT,
                        night2 INT,
                        night3 INT,
                        night4 INT,
                        night5 INT,
                        night6 INT,
                        night7 INT,
                        night8 INT,
                        night9 INT,
                        night10 INT,
                        weather1 VARCHAR(255),
                        weather2 VARCHAR(255),
                        weather3 VARCHAR(255),
                        weather4 VARCHAR(255),
                        weather5 VARCHAR(255),
                        weather6 VARCHAR(255),
                        weather7 VARCHAR(255),
                        weather8 VARCHAR(255),
                        weather9 VARCHAR(255),
                        weather10 VARCHAR(255)
                    );
                """)

                cursor.execute(f"""
                    DELETE FROM {table_name}
                """)

                # cursor.execute(f"""
                #     ALTER TABLE {table_name} CONVERT TO CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
                # """)

                # Загрузка данных в таблицу
                for index, row in df.iterrows():
                    cursor.execute(f"""
                        INSERT INTO {table_name} (
                                            date,
                                            day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,
                                            night1,night2,night3,night4,night5,night6,night7,night8,night9,night10,
                                            weather1,weather2,weather3,weather4,weather5,weather6,weather7,weather8,weather9,weather10)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
                    """, (index,*tuple(row)))  # Передаем значения в виде кортежа

                # Сохранение изменений
                connection.commit()

                print(f"Данные успешно загружены в таблицу '{table_name}'.")

finally:
    connection.close()
    connection_mysql.close()

Данные успешно загружены в таблицу 'Krasnodar_Yandex'.
Данные успешно загружены в таблицу 'Krasnodar_GisMeteo'.
Данные успешно загружены в таблицу 'Ekaterinburg_Yandex'.
Данные успешно загружены в таблицу 'Ekaterinburg_GisMeteo'.
Данные успешно загружены в таблицу 'Moscow_Yandex'.
Данные успешно загружены в таблицу 'Moscow_GisMeteo'.


In [5]:
connection.close()